In [2]:
import pandas as pd

In [4]:
label    = pd.read_csv("./data/covid_labels.csv")
covid_df = pd.read_csv("./data/covid_bert_data.csv")
covid_df["label"] = label["Predicted"]
covid_df.head()

,Unnamed: 0,source,replies,label
0,0,"According to the New York Times, Warner Bros. ...",NaN,0
1,1,Hurricane Hanna has made landfall in Texas.\n\...,@TexasTribune Guess what the cause of death is...,0
2,2,Monkeys on the loose in India with stolen coro...,NaN,1
3,3,NaN,@BelAkinyii When was the last time you washed ...,0
4,4,"“If Trump felt comfortable having it here, the...",@HeidiNBC These Trump fans have a right to att...,1
